In [1]:
%reset -f

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()


Mounted at /content/drive


In [3]:
import ee
ee.Initialize(project='ee-andrewfullhart')

ic = ee.ImageCollection('NASA/NEX-DCP30')

out_file = '/content/drive/MyDrive/Colab Notebooks/NEX_Ensemble_Stats_MaxTemp_MEAN.csv'

study_area = ee.FeatureCollection('users/andrewfullhart/SW_Study_Area')

model_list = ee.List(['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M'])
model_order = ee.List.sequence(0, ee.Number(model_list.size()).subtract(1))

ndays_months = ee.List([31, 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

reducer_list = ee.List([ee.Reducer.percentile([25]),
                        ee.Reducer.percentile([75]),
                        ee.Reducer.min(),
                        ee.Reducer.max(),
                        ee.Reducer.mean()])

reducer_str_list = ee.List(['tasmax_p25', 'tasmax_p75', 'tasmax_min', 'tasmax_max', 'tasmax_mean'])
reducer_order = ee.List.sequence(0, 4)

start_year = 1985
end_year = 2070
global_years_list = ee.List.sequence(start_year, end_year)

modelfilter = ee.Filter.Or(
              ee.Filter.eq('scenario', 'historical'),
              ee.Filter.eq('scenario', 'rcp45'))

ic = ic.filter(modelfilter)

dict_list = []

for year in global_years_list.getInfo():

  print(year)
  start_year = year
  end_year = year
  years_list = ee.List.sequence(start_year, end_year)

  def year_fn(year):

    start = ee.Date.fromYMD(ee.Number(year), 1, 1)
    end = ee.Date.fromYMD(ee.Number(year).add(30), 1, 1)
    year_ic = ic.filterDate(start, end)

    def model_fn(model):

      model_ic = year_ic.filter(ee.Filter.eq('model', model))                   \
                        .select('tasmax')

      def month_fn(month):
        mo_im = model_ic.filter(ee.Filter.calendarRange(month, month,'month'))    \
                      .sum().divide(30).multiply(ee.Number(ndays_months.get(ee.Number(month).subtract(1))))    \
                      .divide(365.25)
        mo_im = mo_im.clip(study_area)
        return mo_im

      ann_im = ee.ImageCollection(order_months.map(month_fn)).sum()
      ann_im = ann_im.expression('(tmax - 273.15)*(1.8)+32', {'tmax':ann_im.select('tasmax')})
      return ann_im

    ann_ic = ee.ImageCollection(model_list.map(model_fn))

    def reducer_fn(re_i):
      re_i = ee.Number(re_i)
      re_im = ann_ic.reduce(reducer_list.get(re_i))
      mean_dict = re_im.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=study_area.geometry(),
        scale=700,
        maxPixels=1e10)
      avg_stat = mean_dict.get(reducer_str_list.get(re_i))
      return avg_stat

    stat_list = reducer_order.map(reducer_fn)

    out_ft = ee.Feature(None,
                      {'year':ee.Number(year),
                        'q25':stat_list.get(0),
                        'q75':stat_list.get(1),
                        'min':stat_list.get(2),
                        'max':stat_list.get(3),
                        'avg':stat_list.get(4)})


    ann_ic_list = ann_ic.toList(model_list.size())
    def reduce_fn(model_i):
      re_im = ee.Image(ann_ic_list.get(ee.Number(model_i)))
      mean_dict = re_im.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=study_area.geometry(),
        scale=500,
        maxPixels=1e10)
      avg_stat = mean_dict.get('tasmax')
      return avg_stat

    stat_list = model_order.map(reduce_fn)

    out_ft = out_ft.set(ee.Dictionary(model_list.zip(stat_list).flatten()))
    return out_ft

  ft_list = years_list.map(year_fn)

  ft = ee.Feature(ft_list.get(0))
  print(ft.getInfo())
  dict_list.append(ft.getInfo())

with open(out_file, 'w') as fo:

  model_list = model_list.getInfo()

  fo.write('yr,min,max,q25,q75,avg,' + ','.join(model_list) + '\n')

  for d in dict_list:
    yr = str(d['properties']['year'])
    min = str(d['properties']['min'])
    max = str(d['properties']['max'])
    q25 = str(d['properties']['q25'])
    q75 = str(d['properties']['q75'])
    avg = str(d['properties']['avg'])

    fo.write(','.join([yr, min, max, q25, q75, avg]))

    for model in model_list:
      fo.write(',' + str(d['properties'][model]))

    fo.write('\n')

1985


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 68.62514983612893, 'BNU-ESM': 68.75939746050493, 'CCSM4': 68.85320175124242, 'CESM1-BGC': 68.67626328869673, 'CNRM-CM5': 68.69111938255601, 'CSIRO-Mk3-6-0': 68.35558988199382, 'CanESM2': 68.59877981241375, 'GFDL-CM3': 68.44879339188437, 'GFDL-ESM2G': 68.78302363159104, 'GFDL-ESM2M': 68.59724480762482, 'IPSL-CM5A-LR': 69.04083812429616, 'IPSL-CM5A-MR': 68.49565035122008, 'MIROC-ESM': 68.42957051214681, 'MIROC-ESM-CHEM': 68.75309948699953, 'MIROC5': 68.67483680832547, 'MPI-ESM-LR': 68.5798140331953, 'MPI-ESM-MR': 68.40238806626286, 'MRI-CGCM3': 67.8985923481884, 'NorESM1-M': 68.44009165790466, 'avg': 68.56731912365872, 'bcc-csm1-1': 68.39783467839759, 'inmcm4': 68.42539705785909, 'max': 69.13095944268852, 'min': 67.89698428761399, 'q25': 68.4267012491859, 'q75': 68.7293505295365, 'year': 1985}}
1986


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 68.60961784259699, 'BNU-ESM': 68.77174138701501, 'CCSM4': 68.91153959482712, 'CESM1-BGC': 68.7229285786393, 'CNRM-CM5': 68.77230791939286, 'CSIRO-Mk3-6-0': 68.44166060171293, 'CanESM2': 68.69382733398707, 'GFDL-CM3': 68.62020583622986, 'GFDL-ESM2G': 68.91252674371425, 'GFDL-ESM2M': 68.59739037733577, 'IPSL-CM5A-LR': 69.10335081272683, 'IPSL-CM5A-MR': 68.63989620982564, 'MIROC-ESM': 68.49035601596617, 'MIROC-ESM-CHEM': 68.89491890969386, 'MIROC5': 68.77580402032973, 'MPI-ESM-LR': 68.52228185339995, 'MPI-ESM-MR': 68.54096788635532, 'MRI-CGCM3': 67.99876376238477, 'NorESM1-M': 68.50683318236354, 'avg': 68.63509412061569, 'bcc-csm1-1': 68.41697482877206, 'inmcm4': 68.40605729446871, 'max': 69.22830178153379, 'min': 67.99698264501109, 'q25': 68.47699318410085, 'q75': 68.80098342178327, 'year': 1986}}
1987


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 68.77311759822506, 'BNU-ESM': 68.87974838574776, 'CCSM4': 68.94062897372471, 'CESM1-BGC': 68.82879081507079, 'CNRM-CM5': 68.81308169076026, 'CSIRO-Mk3-6-0': 68.37081769277547, 'CanESM2': 68.76184959991535, 'GFDL-CM3': 68.71121447345864, 'GFDL-ESM2G': 69.06884258328493, 'GFDL-ESM2M': 68.65624023202612, 'IPSL-CM5A-LR': 69.18665159839773, 'IPSL-CM5A-MR': 68.63259536355106, 'MIROC-ESM': 68.59807390927843, 'MIROC-ESM-CHEM': 68.97184400082288, 'MIROC5': 68.89977107892904, 'MPI-ESM-LR': 68.51861437677667, 'MPI-ESM-MR': 68.58623658531883, 'MRI-CGCM3': 67.98460513550077, 'NorESM1-M': 68.5692689535384, 'avg': 68.69502309393374, 'bcc-csm1-1': 68.4591007556462, 'inmcm4': 68.39736752802747, 'max': 69.28609256219379, 'min': 67.98119128857968, 'q25': 68.50464440580178, 'q75': 68.89390790142492, 'year': 1987}}
1988


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 68.92706155898617, 'BNU-ESM': 68.9317051306645, 'CCSM4': 69.0182704542722, 'CESM1-BGC': 68.88155989042761, 'CNRM-CM5': 68.87930935556119, 'CSIRO-Mk3-6-0': 68.46873998701926, 'CanESM2': 68.86965555771647, 'GFDL-CM3': 68.91286936381475, 'GFDL-ESM2G': 69.07683136290045, 'GFDL-ESM2M': 68.66656834543167, 'IPSL-CM5A-LR': 69.21735363686633, 'IPSL-CM5A-MR': 68.63707213754218, 'MIROC-ESM': 68.66444335275182, 'MIROC-ESM-CHEM': 69.10729641033826, 'MIROC5': 69.05891778243084, 'MPI-ESM-LR': 68.68435120933354, 'MPI-ESM-MR': 68.62718048384647, 'MRI-CGCM3': 67.93963943855921, 'NorESM1-M': 68.59203878084823, 'avg': 68.76228695499324, 'bcc-csm1-1': 68.51802216965596, 'inmcm4': 68.34211681056688, 'max': 69.33789578755037, 'min': 67.93901909595168, 'q25': 68.57293461100635, 'q75': 68.99832981216309, 'year': 1988}}
1989


...

{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 74.11221959482367, 'BNU-ESM': 73.13728543559311, 'CCSM4': 72.74405607183323, 'CESM1-BGC': 72.580768677126, 'CNRM-CM5': 72.72204889242563, 'CSIRO-Mk3-6-0': 73.88721975973384, 'CanESM2': 73.92433037351925, 'GFDL-CM3': 75.60995194849136, 'GFDL-ESM2G': 72.59262280399486, 'GFDL-ESM2M': 71.76829578638085, 'IPSL-CM5A-LR': 74.00684513215192, 'IPSL-CM5A-MR': 74.26378608408066, 'MIROC-ESM': 76.41142028523345, 'MIROC-ESM-CHEM': 76.06527197414904, 'MIROC5': 74.22035174899304, 'MPI-ESM-LR': 72.06796787390206, 'MPI-ESM-MR': 71.74486109938098, 'MRI-CGCM3': 70.69990030434496, 'NorESM1-M': 73.3749958908366, 'avg': 73.29396891231886, 'bcc-csm1-1': 72.3708831598742, 'inmcm4': 70.88127121761421, 'max': 76.60687848473586, 'min': 70.62797400167416, 'q25': 72.26341051220409, 'q75': 74.08943727450958, 'year': 2066}}
2067


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 74.15702678541051, 'BNU-ESM': 73.16661309121255, 'CCSM4': 72.71735128809912, 'CESM1-BGC': 72.62866541578934, 'CNRM-CM5': 72.77923960118655, 'CSIRO-Mk3-6-0': 73.87399460479048, 'CanESM2': 73.96785640898459, 'GFDL-CM3': 75.57010989403085, 'GFDL-ESM2G': 72.54414386718659, 'GFDL-ESM2M': 71.67596435699504, 'IPSL-CM5A-LR': 74.07885310266387, 'IPSL-CM5A-MR': 74.24969517150782, 'MIROC-ESM': 76.2845540442188, 'MIROC-ESM-CHEM': 76.12717591570494, 'MIROC5': 74.24597342592963, 'MPI-ESM-LR': 72.10004985519622, 'MPI-ESM-MR': 71.8241952898463, 'MRI-CGCM3': 70.64475166138037, 'NorESM1-M': 73.43235007706323, 'avg': 73.3021482168441, 'bcc-csm1-1': 72.43379679832897, 'inmcm4': 70.85575628815558, 'max': 76.49932760893958, 'min': 70.5872391489638, 'q25': 72.28037586385798, 'q75': 74.11772473114067, 'year': 2067}}
2068


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 74.20622063971464, 'BNU-ESM': 73.26924879921553, 'CCSM4': 72.67205438533841, 'CESM1-BGC': 72.65859974544775, 'CNRM-CM5': 72.81963716488981, 'CSIRO-Mk3-6-0': 73.91088087514316, 'CanESM2': 74.08131582759778, 'GFDL-CM3': 75.70001101034377, 'GFDL-ESM2G': 72.58721541966692, 'GFDL-ESM2M': 71.85009791328214, 'IPSL-CM5A-LR': 74.01691800372691, 'IPSL-CM5A-MR': 74.180415740974, 'MIROC-ESM': 76.34959946539215, 'MIROC-ESM-CHEM': 76.00835071411052, 'MIROC5': 74.25210001944572, 'MPI-ESM-LR': 72.19009462383573, 'MPI-ESM-MR': 71.85772915010038, 'MRI-CGCM3': 70.67355533914443, 'NorESM1-M': 73.45490724642904, 'avg': 73.32849070864316, 'bcc-csm1-1': 72.35816821131469, 'inmcm4': 70.8141885130807, 'max': 76.56501023908207, 'min': 70.5923041000242, 'q25': 72.29722930210184, 'q75': 74.11648974635202, 'year': 2068}}
2069


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 74.24456313203176, 'BNU-ESM': 73.28173206256372, 'CCSM4': 72.69463923387644, 'CESM1-BGC': 72.64347862265954, 'CNRM-CM5': 72.83390023860767, 'CSIRO-Mk3-6-0': 73.99174025871294, 'CanESM2': 74.12471631132647, 'GFDL-CM3': 75.78626531370249, 'GFDL-ESM2G': 72.6323574446286, 'GFDL-ESM2M': 71.85662537123153, 'IPSL-CM5A-LR': 74.04649906407636, 'IPSL-CM5A-MR': 74.13044982977381, 'MIROC-ESM': 76.3811118651738, 'MIROC-ESM-CHEM': 76.03954878171686, 'MIROC5': 74.32429269942418, 'MPI-ESM-LR': 72.30168054859524, 'MPI-ESM-MR': 71.74068769290575, 'MRI-CGCM3': 70.69262730719295, 'NorESM1-M': 73.46432850349828, 'avg': 73.35011164248591, 'bcc-csm1-1': 72.29218409548069, 'inmcm4': 70.86192037961281, 'max': 76.6216374549433, 'min': 70.61884662281338, 'q25': 72.31959577614431, 'q75': 74.14558062565314, 'year': 2069}}
2070


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 74.24874374812681, 'BNU-ESM': 73.37347422553691, 'CCSM4': 72.6502254231278, 'CESM1-BGC': 72.49976392739315, 'CNRM-CM5': 72.81158895325072, 'CSIRO-Mk3-6-0': 74.03546208638096, 'CanESM2': 74.15636411109647, 'GFDL-CM3': 75.85412854610614, 'GFDL-ESM2G': 72.55564513193606, 'GFDL-ESM2M': 71.797697107822, 'IPSL-CM5A-LR': 74.05385700879185, 'IPSL-CM5A-MR': 74.03634091751795, 'MIROC-ESM': 76.49079320062023, 'MIROC-ESM-CHEM': 76.06242834636178, 'MIROC5': 74.37249647751493, 'MPI-ESM-LR': 72.34411862306426, 'MPI-ESM-MR': 71.90177269828499, 'MRI-CGCM3': 70.8068769048098, 'NorESM1-M': 73.47658402371275, 'avg': 73.36579601261789, 'bcc-csm1-1': 72.25168981315159, 'inmcm4': 70.91466903010362, 'max': 76.72110364352798, 'min': 70.72119383088753, 'q25': 72.26789413697368, 'q75': 74.16456022126616, 'year': 2070}}
